In [ ]:
import pandas as pd
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
from covidvu.pipeline.vujson import JH_CSSE_FILE_CONFIRMED_US, BOATS, _resampleByRegionUS

In [ ]:
from covidvu.pipeline.vujson import COUNTY_NAMES_US

In [ ]:
def _getStateCounts(cases):
    cases = cases.drop(labels=['UID','code3','FIPS', 'Lat', 'Long_','Combined_Key','iso2', 'iso3', 'Admin2', 'Country_Region'], 
             axis=1)
    cases = cases.set_index('Province_State')
    cases = cases.T
    cases.index = pd.to_datetime(cases.index, format='%m/%d/%Y')
    cases = cases.groupby(axis=1, level=0).sum()
    cases = cases.sort_index()
    return cases

In [ ]:
def _getCountyCounts(cases):
    cases = cases.drop(labels=['UID','code3','FIPS', 'Lat', 'Long_','Combined_Key','iso2', 'iso3','Country_Region'], 
             axis=1)
    cases = cases.set_index(['Admin2','Province_State'])
    cases = cases.T
    cases.index = pd.to_datetime(cases.index, format='%m/%d/%Y')
    cases = cases.sort_index()
    return cases

In [ ]:
def _renameCounties(countyName, countyRemapping = COUNTY_NAMES_US):
    if countyName in countyRemapping:
        return countyRemapping[countyName]
    else:
        return countyName

In [ ]:
casesUS = pd.read_csv(JH_CSSE_FILE_CONFIRMED_US)
casesBoats = casesUS[casesUS['Province_State'].isin(BOATS)]
casesNoBoats = casesUS[~casesUS['Province_State'].isin(BOATS)]
casesBoats = _getStateCounts(casesBoats)
casesUSStates = _getStateCounts(casesNoBoats.copy())
casesUSStates['!Total US'] = casesUSStates.sum(axis=1)
casesUSStates = casesUSStates.reindex(sorted(casesUSStates.columns), axis=1)

casesUSCounties = _getCountyCounts(casesNoBoats.copy())
casesUSRegions = _resampleByRegionUS(casesUSStates.copy())

In [ ]:

casesUSCounties.index = casesUSCounties.index.map(lambda s: s.strftime('%Y-%m-%d'))
#casesUSCounties.to_dict()

In [ ]:
from covidvu.pipeline.vujson import dumpJSON

In [ ]:
def castMultiIndexToDict(df):
    return df.groupby(level=0).apply(lambda df: df.xs(df.name).to_dict()).to_dict()

In [ ]:
casesUSCounties.head()

In [ ]:
import numpy as np

In [ ]:
result = {}
for n, ts in casesUSCounties.iteritems():
    countyName, stateName = n
    if isinstance(countyName, float):
        if np.isnan(countyName):
            continue
    result[countyName] = {stateName: ts.to_dict()}
    

In [ ]:
result

In [ ]:
result

In [ ]:
dumpJSON(result, 'test.json')